## In class exercise: Gene Sequence Clustering
Esther Lyon Delsordo <br>
group: Grace Erba, Vlad Kovalanko, Tyler Albrethsen, Jeff Chandler

### Training a Markov model
Load the file genes\_training.p, which is available in this homework archive.  genes\_training.p contains 2000 sequences, with each sequence $\mathbf{s}$ consisting of 20 nucleobases $s_i \in \mathrm{Nu},\; \mathrm{Nu} = \{A,T,G,C\}$.  Each of these sequences is generated from one of two separate Markov processes.  The label (aka class) of the process that generated the sequence is given in the dataset. 

Learn the Markov model for each class given the training data. 
$$
\mathcal{A}_{c,kj} = P(s_{i+1}=\mathrm{Nu}_j|s_{i}=\mathrm{Nu}_k),
$$ 
which is a 4 by 4 matrix.  As a quick sanity check, each row of $\mathcal{A}_c$ should sum to one.  **Using these priors and transition matrices, write a function that generates a new sequence given the class**, i.e. simulates a data point. 

In [2]:
import pickle
import numpy as np

# Define some useful constants
N_nucleobases = 4
N_classes = 2
nucleobases = ['A','T','G','C']

# Load the training data using pickle
sequences,labels = pickle.load(open('genes_training.p','rb'))

# Initialize the class priors and transition matrices
pi_0 = np.zeros((N_classes))
# pi_1 = np.zeros((N_classes))

A_0 = np.zeros((N_nucleobases,N_nucleobases))
A_1 = np.zeros((N_nucleobases,N_nucleobases))

##### Train prior #####

#! Compute class priors
pi_0[0] = sum(labels)
pi_0[1] = len(labels) - sum(labels)

#! Compute unconditional nucleobase probabilities
probs = np.zeros((4))
for i,seq in enumerate(sequences):
    for j,base in enumerate(seq):
        if base=='A':
            probs[0] += 1
        elif base=='T':
            probs[1] += 1
        elif base=='G':
            probs[2] += 1
        elif base=='C':
            probs[3] += 1
        else:
            raise Exception("Base is not in sequence")

probs /= sum(probs)

# Convert from counts to probabilities by normalizing
pi_0/=pi_0.sum()
# pi_1/=pi_1.sum()

##### Train transition matrix #####
ha = {'A':0, 'T':1, 'G':2, 'C':3} # make indices for categorical vars

for s,l in zip(sequences,labels):
    sequence_length = len(s)
    for p in range(sequence_length-1):
        #! s is a length 20 sequence of nucleobases, for all s, count the number of times that a nucleobase 
        #! transitions to another nucleobase and record this information in the appropriate transition matrix (A_0 or A_1)
        if l == 0:
            A_0[ha[s[p-1]],ha[s[p]]] += 1
        if l == 1:
            A_1[ha[s[p-1]],ha[s[p]]] += 1
    
# Convert from counts to probabilities by row normalization
A_0/=A_0.sum(axis=1)[:,np.newaxis]
A_1/=A_1.sum(axis=1)[:,np.newaxis]

##### Generate a synthetic sequence #####
def generate_new_sequence(A,pi,n=20):
    """  
    Arguments:
    A -> Nucleobase transition matrix
    pi -> Prior
    n -> length of sequence to generate
    """
    # Draw from the prior for the first nucleobase
    s = [np.random.choice(nucleobases,p = list(pi))]
    ha = {'A':0, 'T':1, 'G':2, 'C':3}
    #! Write the code that uses the transition matrix to produce a length n sample
    for i in range(1, n):
        s.append(np.random.choice(nucleobases, p = A[ha[s[i-1]]]))
    s = ''.join(s)
    return s


### A Markov classifier
Having the prior and transition probabilities gives you the ability to evaluate the likelihood of a sequence for a given class as:
$$
P(\mathbf{s}) = P(s_1|\mathbf{\pi}_c) \prod_{i=1}^{n-1} P(s_{i+1}|s_{i},\mathcal{A}_c),
$$
where $\mathbf{\pi}_c$ is the class-conditioned prior probability, and $\mathcal{A}_c$ is the class-conditioned transition matrix.  Comparing the computed likelihood for a given sequence between different classes forms the basis of a classifier in a very similar manner to naive Bayes.  The difference this time is that now each random variable depends on the one before it in the sequence, whereas in naive Bayes we assumed that all the random variables were independent.    

Load the file genes\_test.p, which is similar to genes\_training.p.  **For each sequence, compute the likelihood for both classes and assign a label.  Compare this predicted label to the known one, and report the test set accuracy**.  As a point of comparison, my implementation achieved 98.7\% accuracy.

In [3]:
sequences_test,labels_test = pickle.load(open('genes_test.p','rb'))
sequence_probabilities_0 = []
sequence_probabilities_1 = []

for i,s in enumerate(sequences_test):
    #! Write a function that evaluates the probability of class membership for each class by multiplying the 
    #! prior by the likelihood over all symbol transitions
    letters = [*s]
    sequence_probabilities_0.append(probs[ha[letters[0]]]) #put the prior in
    sequence_probabilities_1.append(probs[ha[letters[0]]])
    for letter in range(1,len(letters)):
        sequence_probabilities_0[i] *= A_0[ha[letters[letter-1]], ha[letters[letter]]]
        sequence_probabilities_1[i] *= A_1[ha[letters[letter-1]], ha[letters[letter]]]

prediction = []
for i in range(len(sequences_test)):
    if sequence_probabilities_0[i] > sequence_probabilities_1[i]:
        prediction.append(0)
    else:
        prediction.append(1)
    
is_correct = []
for i in range(len(prediction)):
    if prediction[i] == labels_test[i]:
        is_correct.append(1)
    else:
        is_correct.append(0)
        
# Calculate accuracy of the classifier
prob_of_correct = sum(is_correct)/len(is_correct)
print("Classifier accuracy: ", prob_of_correct)


Classifier accuracy:  0.988


**Turn in a document with the names of those with whom you worked, an example sequence generated by your model for each class, and a statement of your classifier's overall accuracy.**

In [6]:
generate_new_sequence(A_0,probs)

'AATCAATTTAGATCATGAAA'

In [5]:
generate_new_sequence(A_1,probs)

'TGTCCGAGGAACACACCCAG'